_______________________________

With distance model
_______________________________

In [1]:
import gymnasium as gym
import numpy as np
import torch
import os

import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.env_wrappers import EnvBuilder
from src.ppo_agent_v2 import PPOAgent
from src.env_wrappers import GoalObservationWrapper
import src.distance_models as distance_models
from src.utils import trajectories_to_dataset

os.environ['WANDB_API_KEY'] = 'd58c31e07030724bbab6b7e8edc93edacd934c87'

# Check for GPU
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "CPU")

/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchSlide-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchSlide-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchPickAndPlace-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment FetchPickAndPlace-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/user7/.venv/lib/python3.10/site-pack

CUDA available: True
Current device: 0
Device name: Tesla V100-SXM3-32GB


In [2]:
env_id = 'PointMaze_UMaze-v3'
train_episodes = 10000
max_episode_steps = 512
seed = 0

torch.manual_seed(seed); np.random.seed(seed)

# Initialize environment
c = 'c'; U_map = [
    [1, 1, 1, 1, 1],
    [1, c, 0, 0, 1],
    [1, 1, 1, 0, 1],
    [1, c, 0, 0, 1],
    [1, 1, 1, 1, 1]
]
builder = EnvBuilder(env_id=env_id, maze_map=U_map, max_episode_steps=max_episode_steps, seed=seed)
obs_dim, act_dim = builder.get_obs_act_dim()

In [3]:
N_envs = 1
def make_env(env_id, idx, capture_video, run_name, gamma):
    def thunk():
        # Set render_mode to 'rgb_array' if capturing video
        render_mode = "rgb_array" if capture_video and idx == 0 else None
        env = gym.make(env_id, render_mode=render_mode)
        # env = gym.wrappers.FlattenObservation(env)
        env = GoalObservationWrapper(env)
        if capture_video and idx == 0:
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env = gym.wrappers.RecordEpisodeStatistics(env)
        return env
    return thunk

In [4]:
def train_supervised_distance(
        envs,
        env,
        agent,
        distance_model,
        *,
        num_stage: int = 30,
        eval_size: int = 10,
        max_episode_steps: int = 350,
        d_samples: int = 500000,
        en_reward=False,
    ):
        for episode in range(num_stage):
            agent.distance_model = distance_model
            agent.train_ppo(envs, use_distance_shaping = en_reward, verbose=True)
            trajectories = agent.evaluate_ppo(env, num_episodes=eval_size, max_episode_steps=max_episode_steps)

            dataset = trajectories_to_dataset(trajectories, samples=d_samples)
            # sup_model = SupervisedDistanceEstimator(input_dim=4)
            sup_loss = distance_model.train_from_data(dataset, epochs=20, batch_size=16192)          
            print(f"Finish {episode=}, dd loss = {sup_loss}\n")


In [5]:
envs = gym.vector.SyncVectorEnv([make_env(env_id, seed, 0, False, "test")])
env = builder()

In [6]:
EXPERIMENTS = {
    # tag      include_distance_state  use_distance_shaping
    "base"            : (False, False),
    "reward_only"     : (False, True ),
    "state_only"      : (True , False),
    "reward_and_state": (True , True ),
}


In [ ]:
en_state, en_reward = EXPERIMENTS["reward_and_state"]

distance_model = distance_models.SupervisedDistanceEstimator(input_dim=4)
agent = PPOAgent(state_dim=obs_dim, action_dim=act_dim, total_timesteps=10000, distance_model=distance_model, include_distance_state=en_state)
train_supervised_distance(envs, env, agent, distance_model, en_reward=en_reward)

wandb: Currently logged in as: gamershmidt-sofya (gamershmidt-sofya-innopolis-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/user7/.venv/lib/python3.10/site-packages/wandb/analytics/sentry.py:258: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
/home/user7/.venv/lib/python3.10/site-packages/wandb/analytics/sentry.py:258: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


global_step=300, episodic_return=[0.]
global_step=600, episodic_return=[160.]
global_step=900, episodic_return=[0.]
global_step=1200, episodic_return=[0.]
global_step=1500, episodic_return=[204.]
global_step=1800, episodic_return=[51.]
global_step=2100, episodic_return=[182.]
global_step=2400, episodic_return=[0.]
global_step=2700, episodic_return=[0.]
global_step=3000, episodic_return=[0.]
global_step=3300, episodic_return=[0.]
global_step=3600, episodic_return=[0.]
global_step=3900, episodic_return=[0.]
global_step=4200, episodic_return=[0.]
global_step=4500, episodic_return=[0.]
global_step=4800, episodic_return=[0.]
global_step=5100, episodic_return=[0.]
global_step=5400, episodic_return=[0.]
global_step=5700, episodic_return=[0.]
global_step=6000, episodic_return=[0.]
global_step=6300, episodic_return=[0.]
global_step=6600, episodic_return=[101.]
global_step=6900, episodic_return=[0.]
global_step=7200, episodic_return=[0.]
global_step=7500, episodic_return=[0.]
global_step=7800, e

Evaluating: 100%|███████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.02it/s]


Success rate: 0.00%, avg_return: 0.00 ± 0.00


Building dataset: 100%|████████████████████████████████████████████| 10/10 [00:00<00:00, 289.30it/s]


...

Success rate: 30.00%, avg_return: 101.67 ± 32.19


Building dataset: 100%|████████████████████████████████████████████| 10/10 [00:00<00:00, 415.20it/s]


Trajectories processed : 10
Generated samples      : 410711 (train 328568 / eval 82143)


Training, t_loss=0.00: 100%|████████████████████████████████████████| 20/20 [00:00<00:00, 22.90it/s]


eval_losses[0]=0.04221032411536703 -> eval_losses[-1]=0.02362789910416324
Finish episode=5, dd loss = 0.021854931488633156

global_step=300, episodic_return=[0.]
global_step=600, episodic_return=[117.]
global_step=900, episodic_return=[0.]
global_step=1200, episodic_return=[0.]
global_step=1500, episodic_return=[7.]
global_step=1800, episodic_return=[184.]


In [ ]:
agent.save_model(path="runs/ppo_both.cleanrl_model_2")

In [ ]:
distance_model.plot_distance_heatmap(env, source_point=(-1, 1))

In [ ]:
model_path = "results/base/ppo_base.cleanrl_model"
agent = PPOAgent(state_dim=obs_dim, action_dim=act_dim, total_timesteps=10000, distance_model=distance_model, include_distance_state=False)
agent.agent.load_state_dict(torch.load(model_path, map_location="cuda"))
_ = evaluate_ppo(agent, env, num_episodes=100, max_episode_steps=300)


In [ ]:
trajectories = agent.evaluate_ppo(env, num_episodes=100, max_episode_steps=300)

In [ ]:
import gymnasium as gym
import gymnasium_robotics  # If not used elsewhere, you may remove this import.
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
from torch.utils.tensorboard import SummaryWriter
from tqdm import trange
import wandb
import time
import random

def evaluate_ppo(
    agent_cls,
    env: gym.Env,
    *,
    num_episodes: int = 100,
    max_episode_steps: int | None = 170,
    deterministic: bool = True,
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    agent_cls.agent.to(device).eval()

    trajectories   = []
    ep_returns     = []
    success_count  = 0

    for _ in trange(num_episodes, desc="Evaluating", ncols=100):
        obs_np, _ = env.reset()
        obs = torch.as_tensor(obs_np, dtype=torch.float32)
        obs = agent_cls._augment_obs(obs, device)

        episode_traj = []
        for step in range(max_episode_steps):
            ball_x, ball_y = obs.cpu().numpy()[:2]
            episode_traj.append((ball_x, ball_y))

            with torch.no_grad():
                action, *_ = agent_cls.agent.get_action_and_value(obs.unsqueeze(0), None)
            next_obs_np, _, terminated, truncated, info = env.step(action.squeeze(0).cpu().numpy())
            obs = torch.as_tensor(next_obs_np, dtype=torch.float32)
            obs = agent_cls._augment_obs(obs, device)

            if info.get("success", False):
                success_count += 1
                ep_returns.append(max_episode_steps - step)   # reward
                break
            if terminated or truncated:
                break

        trajectories.append(episode_traj)

    # ---------- statistics ----------
    success_rate = success_count / num_episodes
    avg_return   = np.mean(ep_returns) if ep_returns else 0.0
    std_return   = np.std(ep_returns, ddof=1) if len(ep_returns) > 1 else 0.0

    print(f"Success rate: {success_rate:.2%}, "
        f"avg_return: {avg_return:.2f} ± {std_return:.2f}")

    return trajectories


In [ ]:
_ = evaluate_ppo(agent, env, num_episodes=100, max_episode_steps=300)